In [59]:
from pathlib import Path
import os
TRAIN_DIR=Path('/home/abraham/Escritorio/PAE_2/pae2019/CLOCK/CDT_images/train')
TEST_DIR=Path('/home/abraham/Escritorio/PAE_2/pae2019/CLOCK/CDT_images/test')
TRAIN_DIR_DEMENTIA=TRAIN_DIR / "dementia"
TRAIN_DIR_HEALTH=TRAIN_DIR / "health"
TEST_DIR_DEMENTIA=TRAIN_DIR / "dementia"
TEST_DIR_HEALTH=TRAIN_DIR / "health"
train_health_images=[(TRAIN_DIR_HEALTH / i).as_posix() for i in os.listdir(TRAIN_DIR_HEALTH)]
train_dementia_images=[(TRAIN_DIR_DEMENTIA /i).as_posix() for i in os.listdir(TRAIN_DIR_DEMENTIA)]
test_health_images=[(TEST_DIR_HEALTH  /i).as_posix() for i in os.listdir(TEST_DIR_HEALTH)]
test_dementia_images=[(TEST_DIR_DEMENTIA /i).as_posix() for i in os.listdir(TEST_DIR_DEMENTIA)]
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
classifier = Sequential()
classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides=2)) #if stride not given it equal to pool filter size
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides=2))
classifier.add(Flatten())
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=1,activation='sigmoid'))
adam = tensorflow.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
classifier.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))




W1021 11:16:28.118636 140136970573632 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1021 11:16:28.365833 140136970573632 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [38]:
def label_image(img_name):
    if img_name.split("/")[-2]=='health':
        return [1,0]
    elif img_name.split("/")[-2]=='dementia':
        return[0,1]
        
    
    

In [41]:
image_size=50
train_images=train_health_images+train_dementia_images
test_images=test_health_images+test_dementia_images
def train_data_generation():
    training_data=[]
    for image in train_images:
        img_label=label_image(image)
        img=cv2.imread(image,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(image_size,image_size))
        training_data.append([np.array(img),img_label])
    shuffle(training_data)
    return training_data
def test_data_generator():
    test_data=[]
    for image in test_images:
        img_label=label_image(image)
        img=cv2.imread(image,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(image_size,image_size))
        test_data.append([np.array(img),img_label])
    shuffle(test_data)
    return test_data
    
        
        

In [43]:
import tensorflow
from tensorflow.layers.conv import conv_2d,max_pool_2d
from tensorflow.layers.core import input_data,dropout,fully_connected
from tensorflow.layers.estimator import regression

ModuleNotFoundError: No module named 'tensorflow.layers.conv'

In [63]:
train_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                             target_size=(64,64),
                                             batch_size=32,
                                             class_mode='binary')
test_set = test_datagen.flow_from_directory(TEST_DIR,
                                           target_size=(64,64),
                                           batch_size = 32,
                                           class_mode='binary',
                                           shuffle=False)

Found 102 images belonging to 2 classes.
Found 47 images belonging to 2 classes.


In [105]:
classifier.fit_generator(train_set,
                        steps_per_epoch=20, 
                        epochs = 10
                        #callbacks=[tensorboard]
                        );

Epoch 1/10
20/20 [==============================] - 5s 239ms/step - loss: 0.2657 - acc: 0.8902
Epoch 2/10
20/20 [==============================] - 5s 227ms/step - loss: 0.0725 - acc: 0.9627
Epoch 3/10
20/20 [==============================] - 5s 238ms/step - loss: 0.0921 - acc: 0.9706
Epoch 4/10
20/20 [==============================] - 5s 244ms/step - loss: 0.0542 - acc: 0.9863
Epoch 5/10
20/20 [==============================] - 5s 228ms/step - loss: 0.0357 - acc: 0.9902
Epoch 6/10
20/20 [==============================] - 5s 233ms/step - loss: 0.0443 - acc: 0.9863
Epoch 7/10
20/20 [==============================] - 5s 239ms/step - loss: 0.0349 - acc: 0.9863
Epoch 8/10
20/20 [==============================] - 5s 249ms/step - loss: 0.0215 - acc: 0.9980
Epoch 9/10
20/20 [==============================] - 5s 241ms/step - loss: 0.0125 - acc: 1.0000
Epoch 10/10
20/20 [==============================] - 5s 261ms/step - loss: 0.0591 - acc: 0.9784


In [110]:
import numpy as np
from keras.preprocessing import image
predictions=[]
for images in test_images:
    test_image=image.load_img(images,target_size=(64,64))
    test_image=image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    result=classifier.predict(test_image)
    predictions.append(result[0][0])

In [111]:
predictions

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

{'dementia': 0, 'health': 1}